# GATK Tutorial | Hard Filtering | March 2019

This GATK tutorial corresponds to a section of the GATK Workshop _2b. Germline Hard Filtering Tutorial_ worksheet available. The goal is to become familiar with germline variant annotations. The notebook and its paired Python notebook illustrate the following steps. 

- Use GATK to stratify a variant callset against a truthset
- Use R's ggplot2 package to plot the distribution of various annotation values
- Hard-filter based on annotation thresholds and calculate concordance metrics  

### First, make sure the notebook is using an R kernel in the top right corner.
A kernel is a _computational engine_ that executes the code in the notebook. We use this notebook to make plots using R. 

### How to run this notebook:
- **Click to select a gray cell and then pressing SHIFT+ENTER to run the cell.**

- **Write results to `/home/jupyter-user/`. To access the directory, click on the upper-left jupyter icon.**

### Enable reading Google bucket data 

In [ ]:
# Check if data is accessible. The command should list several gs:// URLs.
system("gsutil ls gs://gatk-tutorials/workshop_1702/variant_discovery/data/resources/", intern=TRUE)
system("gsutil ls gs://gatk-tutorials/workshop_1702/variant_discovery/data/intervals/motherHighconf.bed", intern=TRUE)
system("gsutil ls gs://gatk-tutorials/workshop_1702/variant_discovery/data/inputVcfs/", intern=TRUE)

In [ ]:
# If you do not see gs:// URLs listed above, run this cell to install Google Cloud Storage. 
# Afterwards, restart the kernel with Kernel > Restart.
#system("pip install google-cloud-storage", intern=TRUE)

---
## 4. Make density and scatter plots in R and determine filtering thresholds

<span style="color:red">Make sure your kernal is set to R. Go to the menubar and select _Kernel > Change Kernel > R_.</span>

Plotting the density of values for an annotation shows us to see the overall range and distribution of values observed in a callset. In combination with some basic knowledge of what the annotation represents and how it is calculated, this allows us to make a first estimation of value thresholds that segregate FPs from TPs. Plotting the scatter of values for two annotations, one against the other, additionally shows us what tradeoffs we make when setting a threshold on annotation values individually. 



### A. Load R libraries, plotting functions and data

Don't worry if you don't know how to read the R script below. Also, you can ignore the red boxes that appear, e.g. stating `as ‘lib’ is unspecified`. 

In [ ]:
# plotting.R script loads ggplot and gridExtra libraries and defines functions to plot variant annotations 

library(ggplot2)
install.packages("gridExtra")
library(gridExtra)

get_legend<-function(myggplot){
  tmp <- ggplot_gtable(ggplot_build(myggplot))
  leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
  legend <- tmp$grobs[[leg]]
  return(legend)
}


# Function for making density plots of a single annotation
makeDensityPlot <- function(dataframe, xvar, split, xmin=min(dataframe[xvar], na.rm=TRUE), xmax=max(dataframe[xvar], na.rm=TRUE), alpha=0.5) {
  
  if(missing(split)) {
    return(ggplot(data=dataframe, aes_string(x=xvar)) + xlim(xmin,xmax) + geom_density() )
  }
  else {
    return(ggplot(data=dataframe, aes_string(x=xvar, fill=split)) + xlim(xmin,xmax) + geom_density(alpha=alpha) )
  }
}

# Function for making scatter plots of two annotations
makeScatterPlot <- function(dataframe, xvar, yvar, split, xmin=min(dataframe[xvar], na.rm=TRUE), xmax=max(dataframe[xvar], na.rm=TRUE), ymin=min(dataframe[yvar], na.rm=TRUE), ymax=max(dataframe[yvar], na.rm=TRUE), ptSize=1, alpha=0.6) {
  if(missing(split)) {
    return(ggplot(data=dataframe) + aes_string(x=xvar, y=yvar) + xlim(xmin,xmax) + ylim(ymin,ymax) + geom_point(size=ptSize, alpha=alpha) )
  }
  else {
    return(ggplot(data=dataframe) + aes_string(x=xvar, y=yvar) + aes_string(color=split) + xlim(xmin,xmax) + ylim(ymin,ymax) + geom_point(size=ptSize, alpha=alpha) )
  }
}

# Function for making scatter plots of two annotations with marginal density plots of each
makeScatterPlotWithMarginalDensity <- function(dataframe, xvar, yvar, split, xmin=min(dataframe[xvar], na.rm=TRUE), xmax=max(dataframe[xvar], na.rm=TRUE), ymin=min(dataframe[yvar], na.rm=TRUE), ymax=max(dataframe[yvar], na.rm=TRUE), ptSize=1, ptAlpha=0.6, fillAlpha=0.5) {
  empty <- ggplot()+geom_point(aes(1,1), colour="white") +
    theme(
      plot.background = element_blank(), 
      panel.grid.major = element_blank(), 
      panel.grid.minor = element_blank(), 
      panel.border = element_blank(), 
      panel.background = element_blank(),
      axis.title.x = element_blank(),
      axis.title.y = element_blank(),
      axis.text.x = element_blank(),
      axis.text.y = element_blank(),
      axis.ticks = element_blank()
    )
  
  if(missing(split)){
    scatter <- ggplot(data=dataframe) + aes_string(x=xvar, y=yvar) + geom_point(size=ptSize, alpha=ptAlpha) + xlim(xmin,xmax) + ylim(ymin,ymax) 
    plot_top <- ggplot(data=dataframe, aes_string(x=xvar)) + geom_density(alpha=fillAlpha) + theme(legend.position="none") + xlim(xmin,xmax) 
    plot_right <- ggplot(data=dataframe, aes_string(x=yvar)) + geom_density(alpha=fillAlpha) + coord_flip() + theme(legend.position="none") + xlim(ymin,ymax) 
  } 
  else{
    scatter <- ggplot(data=dataframe) + aes_string(x=xvar, y=yvar) + geom_point(size=ptSize, alpha=ptAlpha, aes_string(color=split)) + xlim(xmin,xmax) + ylim(ymin,ymax) 
    plot_top <- ggplot(data=dataframe, aes_string(x=xvar, fill=split)) + geom_density(alpha=fillAlpha) + theme(legend.position="none") + xlim(xmin,xmax) 
    plot_right <- ggplot(data=dataframe, aes_string(x=yvar, fill=split)) + geom_density(alpha=fillAlpha) + coord_flip() + theme(legend.position="none") + xlim(ymin,ymax) 
  }
  legend <- get_legend(scatter)
  scatter <- scatter + theme(legend.position="none")
  temp <- grid.arrange(plot_top, legend, scatter, plot_right, ncol=2, nrow=2, widths=c(4,1), heights=c(1,4))
  return(temp)
}

In [ ]:
# Call the readr library and use its read_delim function to load motherSNP.giab.txt into the motherSNP.giab object.
library(readr)
motherSNP.giab <- read_delim("/home/jupyter-user/motherSNP.giab.txt","\t", 
              escape_double = FALSE, col_types = cols(giab.callsets = col_character()), trim_ws = TRUE)

In [ ]:
# Rename the 'giab.callsets' column to 'set'.
names(motherSNP.giab)[names(motherSNP.giab) == 'giab.callsets'] <- 'set'

---

For reference, here are some basic filtering thresholds to improve upon.

- -filter "QD < 2.0"
- -filter "QUAL < 30.0"
- -filter "SOR > 3.0"
- -filter "FS > 60.0"
- -filter "MQ < 40.0"
- -filter "MQRankSum < -12.5 
- -filter "ReadPosRankSum < -8.0"

---

### B. Make a density plot for QUAL with the `makeDensityPlot` function

Iteratively improve the plot by modifying `qual`. Here are some suggestions to start.
- B = makeDensityPlot(motherSNP.giab, "QUAL")
- B = makeDensityPlot(motherSNP.giab, "QUAL", xmax=10000)
- B = makeDensityPlot(motherSNP.giab, "QUAL", xmax=10000, split="set")

> _How does the density distribution relate to what the annotation represents? Can we find some clues of what might distinguish good vs. bad variants?_
> _When we plot the split version, can we see a clear difference between the set distributions? What does that tell us?_

In [ ]:
# B = makeDensityPlot(motherSNP.giab, "QUAL")
B = makeDensityPlot(motherSNP.giab, "QUAL")


In [ ]:
# Plot 'B'
B

### C. Make a QD (QualByDepth) density plot

QD puts the variant confidence QUAL score into perspective by normalizing for the amount of coverage available. Because each read contributes a little to the QUAL score, variants in regions with deep coverage can have artificially inflated QUAL scores, giving the impression that the call is supported by more evidence than it really is. To compensate for this, we normalize the variant confidence by depth, which gives us a more objective picture of how well supported the call is.

> _What do the peaks represent?_

In [ ]:
# C = makeDensityPlot(motherSNP.giab, "QD")
# Change up the parameters, e.g. add 'split="set"', examine RankSums, FS and SOR
C = makeDensityPlot(motherSNP.giab, 
                    "QD")

In [ ]:
C

### D. Make a scatterplot of QD vs. DP using the `makeScatterPlot` function

The DP (depth) here refers to the unfiltered count of reads at the site level (INFO). An identically named annotation exists at the sample level (FORMAT) that refers to the count of reads that passed the caller's internal quality control metrics for the sample. 

> What is the relationship between DP and QUAL? How does high-depth correlate with true positives?

In [ ]:
# D = makeScatterPlot(motherSNP.giab, "QD", "DP", split="set")
# Play with the axis limits to zoom in on subsets of the data, e.g. by adding ymax=1000.
D = makeScatterPlot(motherSNP.giab, 
                    "QD", "DP")

In [ ]:
D

### E. Make a scatterplot winged by marginal density plots

The `makeScatterPlotWithMarginalDensity` function defines and plots. The `ptAlpha` parameter changes the transparency of the points. 

> _When plotting two annotations, does the combination of the two tell us anything more than either did separately?_

- Try adjusting the parameters.
- Substitute in other annotations. For example, the following recreates the plot on the front page of the tutorial worksheet.

```
F = makeScatterPlotWithMarginalDensity(motherSNP.giab, "QUAL", "DP", split="set", xmax=10000, ymax=100, ptSize=0.5, ptAlpha=0.05)
```




In [ ]:
# E = makeScatterPlotWithMarginalDensity(motherSNP.giab, "QD", "DP", split="set", ymax=250, ptSize=0.5, ptAlpha=0.2)
E = makeScatterPlotWithMarginalDensity(motherSNP.giab, 
                                       "QD", "DP", 
                                       split="set", 
                                       ymax=250, 
                                       ptSize=0.5, ptAlpha=0.2)

In [ ]:
# Blank cell for free use. Add additional cells with Menu > Insert.
# Change the cell type with Cell > Cell Type.
# Delete a cell with Edit > Delete Cells.

---
## 5. Apply filters with VariantFiltration and evaluate results

<span style="color:blue">Go back to the Python notebook now to continue your work.</span>